In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df=pd.read_csv('/content/sample_data/letter-recognition.csv')
diag_map = {'A': 1.0, 'B': -1.0,'C': 1.0, 'D': -1.0,'E': 1.0, 'F': -1.0,'G': 1.0, 'H': -1.0,'I': 1.0, 'J': -1.0,'K': 1.0, 'L': -1.0,'M': 1.0, 'N': -1.0
            ,'O': 1.0, 'P': -1.0,'Q': 1.0, 'R': -1.0,'S': 1.0, 'T': -1.0,'U': 1.0, 'V': -1.0,'W': 1.0, 'X': -1.0,
            'Y': 1.0, 'Z': -1.0}
df['letter'] = df['letter'].map(diag_map)
df=df.sample(frac=1)
df


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
6264,-1.0,2,10,3,8,1,13,2,9,4,14,4,12,1,6,0,8
2387,-1.0,8,12,6,6,4,9,5,4,5,10,5,9,5,7,7,10
4718,-1.0,3,5,5,6,4,11,5,3,2,8,4,9,4,8,5,11
13717,1.0,5,7,7,6,9,7,6,4,5,7,5,8,9,9,7,8
19358,-1.0,6,9,8,7,4,3,13,4,5,13,9,4,1,10,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,-1.0,4,8,6,6,4,8,7,5,7,11,5,5,3,8,3,8
15628,-1.0,4,10,5,8,4,6,4,1,7,8,2,10,0,6,3,8
2024,1.0,7,11,8,8,5,6,6,7,8,10,7,11,2,10,5,9
10555,1.0,2,1,2,1,0,8,6,11,4,7,12,8,3,10,0,8


In [ ]:
m=5000
X=df.drop('letter',axis=1)
y=df['letter']
print(X.head())

       xbox   ybox   width   height  ...  xedge   xedgey  yedge   yedgex
19770      1      0       1       0  ...       2       8       0       8
13344      6      9       7       8  ...       3       9       8       7
6843       5      5       6       3  ...       7      11       1       7
5187       3      5       4       7  ...       0       8       0       8
13701      6     12       5       7  ...       4       9       5      13

[5 rows x 16 columns]


In [ ]:
X_train=X[:m]
y_train=y[:m]
X_test=X[m:m+2000]
y_test=y[m:m+2000]

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred=clf.predict(X_test)

**RESULTS SVM WITHOUT MARKOV SAMPLING**

In [ ]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, y_pred)))

accuracy on test dataset: 0.7245
recall on test dataset: 0.7591463414634146
precision on test dataset: 0.7591463414634146


In [ ]:
N1=1000
X_val=X[:N1]
y_val=y[:N1]
print(y_val.shape)

(1000,)


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_val, y_val)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pre=clf.predict(X_test)
print(y_pre)

[ 1. -1.  1. ... -1.  1.  1.]


In [ ]:
def Cost(Y_pred, Y):
    # calculate hinge loss
    if Y_pred*Y>1.0:
        return 0
    return 1.0-Y_pred*Y

In [ ]:
X=X.to_numpy()
y=y.to_numpy()

In [ ]:
k=5
q=1/1.2
print(q)
print(m)

0.8333333333333334
5000


In [ ]:
import math
mp=mn=0
ct=0
X_tr=[]
y_tr=[]
X_t=X[0].reshape(1,-1)
y_t=y[0]
y_pred_t=clf.predict(X_t)
if m%2==0 and y_t==1.0:
    mp+=1
elif m%2==0 and y_t==-1.0:
    mn+=1
for i in range(1,m):
    X_z=X[i].reshape(1,-1)
    y_z=y[i]
    y_pred_z=clf.predict(X_z)
    P=math.exp(-Cost(y_pred_z,y_z))/math.exp(-Cost(y_pred_t,y_t))
    if ct>=k:
        Pdd=q*P
        if(Pdd>0.5):
            X_tr.append(X_z[0])
            y_tr.append(y_z)
            ct+=1
        else:
            ct=0
    elif P==1.0 and y_z==y_t:
        Pd=math.exp(y_z*y_pred_z)/math.exp(y_t*y_pred_t)
        if Pd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z)
            ct+=1
        else:
            ct=0
    elif P<1.0 or P==1.0 and y_z*y_t==-1.0:
        if P>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z)
            ct+=1
        else:
            ct=0
    elif P>1.0:
        X_tr.append(X_z[0])
        y_tr.append(y_z)
        ct+=1
    if y_t==1.0:
        mp+=1
    else:
        mn+=1
    y_t=y_z
    X_t=X_z
    y_pred_t=y_pred_z
    if mp<m/2 or mn<m/2:
        continue
    else:
        break

In [ ]:
print(mp,mn)

2536 2464


In [ ]:
print(len(X_tr))
X_tr=np.array(X_tr)
y_tr=np.array(y_tr)

4014


In [ ]:
print(X_tr.shape,y_tr.shape[0])

(4014, 16) 4014


In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_tr, y_tr)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred=clf.predict(X_test)
# for i in range(X_te.shape[0]):
#     pre=clf.predict(X_te[i].reshape(1,-1))
#     y_pred.append(pre[0])
# y_pred=np.array(y_pred)

In [ ]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, y_pred)))

accuracy on test dataset: 0.73
recall on test dataset: 0.7926829268292683
precision on test dataset: 0.7926829268292683
